In [1]:
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [3]:
import allel
import numpy as np
import pandas as pd
import seaborn as sns

## PCA using sci-kit learn

### PCA using vcf files

Loading in the vcf files as well as the metadata file

In [12]:
chr7_females = allel.read_vcf('../steps/recode_vcf/chr7_females.recode.vcf')

In [13]:
chrX_females = allel.read_vcf('../steps/recode_vcf/chrX_females.recode.vcf')

In [14]:
chr7_males = allel.read_vcf('../steps/recode_vcf/chr7_males.recode.vcf')

In [15]:
chrX_males = allel.read_vcf('../../../../data/haploidified_chrX_males/hap_chrX_males.vcf.gz')

In [16]:
meta_data_samples = pd.read_table("../data/metadata.txt", sep=" ")

In [17]:
meta_data_females = meta_data_samples[meta_data_samples['Sex'] == 'F']
meta_data_males = meta_data_samples[(meta_data_samples['Sex'] == 'M') & (meta_data_samples['Genus'] == 'Papio')]

Function for converting vcf file into df to be used for PCA.

In [4]:
def vcf2df(vcf):
    gt = allel.GenotypeArray(vcf['calldata/GT'])
    ac = gt.count_alleles()
    
    flt = (ac.max_allele() == 1) & (ac[:, :2].min(axis=1) > 1)
    gf = gt.compress(flt, axis=0)
    
    gn = gf.to_n_alt()
    
    return gn

Function for pruning the df.

In [5]:
def ld_prune(gn, size, step, threshold=.1, n_iter=1):
    for i in range(n_iter):
        loc_unlinked = allel.locate_unlinked(gn, size=size, step=step, threshold=threshold)
        n = np.count_nonzero(loc_unlinked)
        n_remove = gn.shape[0] - n
        print('iteration', i+1, 'retaining', n, 'removing', n_remove, 'variants')
        gn = gn.compress(loc_unlinked, axis=0)
    return gn

Function for performing PCA analysis.

In [6]:
def perform_pca(df, vcf, components, pop_df):
    pipeline = make_pipeline(StandardScaler(), sklearnPCA(n_components=components))
    data_transf = pipeline.fit_transform(df.T)
    
    principal_df = pd.DataFrame(data = data_transf, 
                                columns = ['PC{}'.format(i) for i in range(1, components + 1)],
                                index=vcf['samples'])
    
    pop_df = pop_df.set_index('PGDP_ID')
    
    principal_df.index = principal_df.index.str.replace('Sci_', '', regex=True) # Removing instances where individuals have the prefix 'Sci'
    
    final_df = pd.merge(principal_df, pop_df['C_origin'], left_index=True, right_index=True)

    pca = pipeline.get_params()['pca']
    
    return final_df, pca

Function for plotting two pcs.

In [7]:
def plot_pca(df, pca, pop, chrom, pc_xaxis, pc_yaxis):
    pc1_var = (pca.explained_variance_ratio_[pc_xaxis-1]*100).round(decimals=2)
    pc2_var = (pca.explained_variance_ratio_[pc_yaxis-1]*100).round(decimals=2)
    
    fig, ax = plt.subplots(figsize = (6,5))
    
    groups = df.groupby('C_origin')
    for name, group in groups:
        plt.scatter(group['PC{}'.format(pc_xaxis)], group['PC{}'.format(pc_yaxis)], label=name)
        
    ax.set_xlabel('Principal Component {} ({}%)'.format(pc_xaxis, pc1_var), fontsize = 10)
    ax.set_ylabel('Principal Component {} ({}%)'.format(pc_yaxis, pc2_var), fontsize = 10)
    ax.text(s = 'Chromosome {}'.format(chrom), size=20, rotation=0, ha = 'center', x=0.5,y=1.06,transform=ax.transAxes)
#     plt.title('{}'.format(pop), fontsize = 20, rotation='vertical',x=-0.22,y=0.35)
    
    
    ax.legend(bbox_to_anchor=(1.01, 0.75), loc=2)
#     ax.get_legend().remove()
    
    
#     fig.tight_layout()
    plt.show()
    fig.savefig('../figures/PC{}_{}_{}_{}.pdf'.format(pc_xaxis, pc_yaxis, pop, chrom), bbox_inches='tight')

Function for plotting the variance explained ratio of each of the pcs.

In [8]:
def var_expl(pca):
    fig, ax = plt.subplots(figsize = (6,5))
    
    ax.bar(x = range(1, 11), height = pca.explained_variance_ratio_)
    
    ax.set_xlabel('Principal Components', fontsize = 10)
    ax.set_ylabel('Variance explained', fontsize = 10)
    ax.set_title('Distribution of variance explained', fontsize = 15)
    ax.set_ylim(0, max(pca.explained_variance_ratio_)*2)

    plt.show()

#### Females
##### Chromosome 7

In [ ]:
females_7 = vcf2df(chr7_females)

In [ ]:
pruned_females_7 = ld_prune(females_7, size=200, step=20, threshold=.1, n_iter=1)

In [ ]:
pca_chr7_females = perform_pca(pruned_females_7, chr7_females, 10, meta_data_females)

In [ ]:
var_expl(pca_chr7_females[1])

In [ ]:
plot_pca(pca_chr7_females[0], pca_chr7_females[1], 'Females', 7, 1, 2)

In [ ]:
plot_pca(pca_chr7_females[0], pca_chr7_females[1], 'Females', 7, 2, 3)

##### Chromosome X

In [ ]:
females_X = vcf2df(chrX_females)

In [ ]:
pruned_females_X = ld_prune(females_X, size=200, step=20, threshold=.1, n_iter=1)

In [ ]:
pca_chrX_females = perform_pca(pruned_females_X, chrX_females, 10, meta_data_females)

In [ ]:
var_expl(pca_chrX_females[1])

In [ ]:
plot_pca(pca_chrX_females[0], pca_chrX_females[1], 'Females', 'X', 1, 2)

In [ ]:
plot_pca(pca_chrX_females[0], pca_chrX_females[1], 'Females', 'X', 2, 3)

#### Males
##### Chromosome 7

In [ ]:
males_7 = vcf2df(chr7_males)

In [ ]:
pruned_males_7 = ld_prune(males_7, size=200, step=20, threshold=.1, n_iter=1)

In [ ]:
pca_chr7_males = perform_pca(pruned_males_7, chr7_males, 10, meta_data_males)

In [ ]:
var_expl(pca_chr7_males[1])

In [ ]:
plot_pca(pca_chr7_males[0], pca_chr7_males[1], 'Males', 7, 1, 2)

In [ ]:
plot_pca(pca_chr7_males[0], pca_chr7_males[1], 'Males', 7, 2, 3)

##### Chromosome X

In [ ]:
males_X = vcf2df(chrX_males)

In [ ]:
pruned_males_X = ld_prune(males_X, size=200, step=20, threshold=.1, n_iter=1)

In [ ]:
pca_chrX_males = perform_pca(pruned_males_X, chrX_males, 10, meta_data_males)

In [ ]:
var_expl(pca_chrX_males[1])

In [ ]:
plot_pca(pca_chrX_males[0], pca_chrX_males[1], 'Males', 'X', 1, 2)

In [ ]:
plot_pca(pca_chrX_males[0], pca_chrX_males[1], 'Males', 'X', 2, 3)

### PCA using chromopainter output

In [18]:
def plot_fs_pca(df, pca, pc_xaxis, pc_yaxis, chrom, pop):
    pc1_var = (pca.explained_variance_ratio_[pc_xaxis-1]*100).round(decimals=2)
    pc2_var = (pca.explained_variance_ratio_[pc_yaxis-1]*100).round(decimals=2)

    fig, ax = plt.subplots(figsize = (6,5))

    groups = df.groupby('C_origin')
    for name, group in groups:
        ax.scatter(group['PC{}'.format(pc_xaxis)], group['PC{}'.format(pc_yaxis)], label=name)

    ax.set_xlabel('Principal Component {} ({}%)'.format(pc_xaxis, pc1_var), fontsize = 10)
    ax.set_ylabel('Principal Component {} ({}%)'.format(pc_yaxis, pc2_var), fontsize = 10)
    ax.text(s = 'Chromosome {}'.format(chrom), size=20, rotation=0, ha = 'center', x=0.5,y=1.06,transform=ax.transAxes)
    plt.title('{}'.format(pop), fontsize = 20, rotation='vertical',x=-0.22,y=0.35)
    
    ax.legend(bbox_to_anchor=(1.01, 0.75), loc=2)
#     ax.get_legend().remove()

#     fig.tight_layout()
    fig.savefig('../figures/fs_pc{}_{}_{}_{}.pdf'.format(pc_xaxis, pc_yaxis, pop, chrom), bbox_inches='tight')
    plt.show()

#### Data

In [19]:
fs_out_malesX = pd.read_table("../steps/finestructure/test_run_linked.chunkcounts.out", sep=" ", header = 1, index_col = 0)
fs_out_males7 = pd.read_table("../steps/finestructure/male_chr7_unlinked_unlinked.chunkcounts.out", sep=" ", header = 1, index_col = 0)
fs_out_femalesX = pd.read_table("../steps/finestructure/female_unlinked_unlinked.chunkcounts.out", sep=" ", header = 1, index_col = 0)
fs_out_females7 = pd.read_table("../steps/finestructure/female_chr7_unlinked_unlinked.chunkcounts.out", sep=" ", header = 1, index_col = 0)

##### Females

In [20]:
fs_females7 = perform_pca(fs_out_females7, chr7_females, 10, meta_data_females) 

In [ ]:
var_expl(fs_females7[1])

In [ ]:
plot_fs_pca(fs_females7[0], fs_females7[1], 1, 2)

In [ ]:
fs_femalesX = perform_pca(fs_out_femalesX, chrX_females, 10, meta_data_females) 

In [ ]:
var_expl(fs_femalesX[1])

In [ ]:
plot_fs_pca(fs_femalesX[0], fs_femalesX[1], 1, 2, 'X', 'Females')

##### Males

In [ ]:
fs_males7 = perform_pca(fs_out_males7, chr7_males, 10, meta_data_males) 

In [ ]:
var_expl(fs_males7[1])

In [ ]:
plot_fs_pca(fs_males7[0], fs_males7[1], 1, 2, '7', 'Males')

In [ ]:
fs_malesX = perform_pca(fs_out_malesX, chrX_males, 10, meta_data_males) 

In [ ]:
var_expl(fs_malesX[1])

In [ ]:
plot_fs_pca(fs_malesX[0], fs_malesX[1], 1, 2, 'X', 'Males')

In [ ]:
plot_fs_pca(fs_malesX[0], fs_malesX[1], 2, 3)